# Exercise 4: Hypertension patients

We want to investigate hypertension patients based on the NCD download data. 

Questions we want to investigate: 

* What is the gender breakdown of Hypertension patients
* Is there a gender difference in the number of controlled hypertension patients
* Want to look at the geographic distribution of the fraction of controlled hypertension patients
* (Bonus) Build a linear regression model to investigate which factors impact BP in hypertension patients

This exercise is using Pandas to read in data downloaded from the website. 

In [3]:
import pandas as pd

We import the data from an excel file

In [4]:
data = pd.read_excel("../../../non_communicable_diseases.xlsx")

We get all the hypertension patients as follows: 

In [5]:
hypertension_data = data[data["Common Name"] == "Hypertension"]

In [6]:
# We calculate controlled patients as patients with systolic BP less than 150
hypertension_data["controlled"] = (hypertension_data["Systolic BP"] < 150).apply(int)

/home/gunnar/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Now you can try to investigate the gender distribution of controlled and not controlled patients

In [4]:
hypertension_data["Gender"].value_counts()

female    47098
male      30326
Name: Gender, dtype: int64

One useful function is to create a pivot table as in the example below:

In [6]:
hypertension_data.pivot_table(index=["Gender","controlled"], aggfunc="count")["Clinic"]

Gender  controlled
female  0             42243
        1              4855
male    0             27091
        1              3235
Name: Clinic, dtype: int64

If we want to calculate the fraction of controlled patients for each gender the easiest thing is to iterate over each group as below. 

In [7]:
for name, group in hypertension_data.groupby("Gender"):
    fraction = group["controlled"].sum() / len(group)
    print(name, fraction)

female 0.529661556754
male 0.532381454857


You can now try the same with using the Region variable instead of Gender. To see the geographic distribution

In [8]:
hypertension_data.pivot_table(index=["Region","controlled"], aggfunc="count")["Clinic"]


Region    controlled
Ajloun    0               774
          1              2120
Amman     0              6221
          1              7533
Aqaba     0               374
          1               786
Balqa     0              4563
          1              6780
Irbid     0             10366
          1              6767
Jarash    0              1094
          1              2098
Karak     0              3043
          1              3877
Ma'an     0               110
          1               264
Madaba    0              1401
          1              2191
Mafraq    0              1274
          1              1277
Petra     0               344
          1               329
Ramtha    0              1479
          1               603
Tafeileh  0              1245
          1               523
Zarqa     0              4166
          1              6004
Name: Clinic, dtype: int64

In [9]:
for name, group in hypertension_data.groupby("Region"):
    fraction = group["controlled"].sum() / len(group)
    print(name, fraction)

Ajloun 0.732550103663
Amman 0.547695215937
Aqaba 0.677586206897
Balqa 0.597725469453
Irbid 0.394968773712
Jarash 0.657268170426
Karak 0.560260115607
Ma'an 0.705882352941
Madaba 0.609966592428
Mafraq 0.500588004704
Petra 0.488855869242
Ramtha 0.289625360231
Tafeileh 0.295814479638
Zarqa 0.590363815143


As a bonus we can try to create a linear regression model to estimate a patients BP from other factors. To this we use the statsmodel packages as below

In [12]:
import statsmodels.formula.api as smf

In [10]:
# To use this package properly we need to make sure all the column names are a bit easier to deal with 
from slugify import slugify
smf_data =hypertension_data.rename(columns=lambda x: slugify(x).replace("-", "_"))
smf_data.columns

Index(['visit_type', 'visit_date', 'epi_week', 'vist_day', 'vist_month',
       'visit_year', 'region', 'district', 'clinic', 'status',
       ...
       'dose_of_medicine_3_prescribed',
       'availability_of_medicine_3_prescribed',
       'name_of_medicine_4_prescribed', 'dose_of_medicine_4_prescribed',
       'availability_of_medicine_4_prescribed',
       'name_of_medicine_5_prescribed', 'dose_of_medicine_5_prescribed',
       'availability_of_medicine_5_prescribed', 'uuid', 'controlled'],
      dtype='object', length=129)

We can now create our first regression model of modelling the age and gender effect on BP. We specify our model using a formula language `systolic_bp ~ age_years + C(gender)`. The C(gender), means that gender is treated as a categorical factor. We can see the result of this regression analysis below

In [13]:
smf.ols("systolic_bp ~ age_years + C(gender)", data=smf_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            systolic_bp   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     51.82
Date:                Fri, 22 Sep 2017   Prob (F-statistic):           3.30e-23
Time:                        10:49:25   Log-Likelihood:            -2.3827e+05
No. Observations:               55148   AIC:                         4.765e+05
Df Residuals:                   55145   BIC:                         4.766e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           132.5812      0.391    338.927      0.000     131.815     133.348
C(gender)[T.male]     0.2940      0.159      1.852      0.064      -0.017       0.605
age_years             0.0637      0.006      9.944      0.000       0.051       0.076
==============================================================================
Omnibus:                     4387.054   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7684.979
Skew:                           0.584   Prob(JB):                         0.00
Kurtosis:                       4.408   Cond. No.                         306.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

You can now extedn this model to try to understand what the impact of for example Region, BMI or visit type has on the BP of the patients

In [14]:
smf.ols("systolic_bp ~ age_years + C(gender) + C(region) + bmi + C(visit_type)", data=smf_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            systolic_bp   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     83.64
Date:                Fri, 22 Sep 2017   Prob (F-statistic):          4.13e-302
Time:                        10:50:40   Log-Likelihood:            -1.4142e+05
No. Observations:               32915   AIC:                         2.829e+05
Df Residuals:                   32896   BIC:                         2.830e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                   130.6391      0.670    195.129      0.000     129.327     131.951
C(gender)[T.male]             0.1630      0.202      0.809      0.419      -0.232       0.558
C(region)[T.Amman]            1.1614      0.453      2.566      0.010       0.274       2.048
C(region)[T.Aqaba]            8.6811      1.151      7.545      0.000       6.426      10.936
C(region)[T.Balqa]            3.0789      0.449      6.856      0.000       2.199       3.959
C(region)[T.Irbid]            4.2972      0.475      9.042      0.000       3.366       5.229
C(region)[T.Jarash]           3.6779      0.643      5.721      0.000       2.418       4.938
C(region)[T.Karak]            3.2035      0.487      6.575      0.000       2.249       4.158
C(region)[T.Ma'an]           -7.0833      1.347     -5.257      0.000      -9.724      -4.442
C(region)[T.Madaba]           0.7075      0.561      1.260      0.208      -0.393       1.808
C(region)[T.Mafraq]           0.7922      0.813      0.974      0.330      -0.801       2.386
C(region)[T.Petra]           11.4492      1.282      8.932      0.000       8.937      13.962
C(region)[T.Ramtha]           5.9535      0.810      7.354      0.000       4.367       7.540
C(region)[T.Tafeileh]        12.0997      0.890     13.590      0.000      10.355      13.845
C(region)[T.Zarqa]            8.5555      0.485     17.627      0.000       7.604       9.507
C(visit_type)[T.referral]    -4.4065      2.657     -1.658      0.097      -9.614       0.801
C(visit_type)[T.return]      -4.9424      0.212    -23.260      0.000      -5.359      -4.526
age_years                     0.0777      0.008      9.570      0.000       0.062       0.094
bmi                           0.0159      0.006      2.570      0.010       0.004       0.028
==============================================================================
Omnibus:                     2709.408   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5045.599
Skew:                           0.580   Prob(JB):                         0.00
Kurtosis:                       4.528   Cond. No.                     1.86e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.86e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""